## BERT의 문장 임베딩(SBERT)을 이용한 한국어 챗봇
SBERT 를 이용하여 문장 (문서) 임베딩을 얻을 수 있는 패키지인 sentence_transformers 를 사용하여 쉽고 간단하게 한국어 챗봇을 구현해봅시다. 실습에 앞서 sentence_transformers 를 설치합니다.

In [1]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

트랜스포머를 이용한 한국어 챗봇 실습에서 사용했던 데이터를 그대로 사용합니다.

In [2]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


문장 임베딩을 얻기 위해서 사전 훈련된 BERT 를 로드합니다. 여기서는 한국어도 포함되어 학습된 다국
어 모델을 로드합니다.

In [4]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

모델의 이름은’xlm‐r‐100langs‐bert‐base‐nli‐stsb‐mean‐tokens’ 인데 이름이 의미하는 바는 100 가지 언
어를 지원 (한국어 포함) 하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS‐B 데이터로 학습되었
으며, 문장 표현을 얻기 위해서는 평균 풀링 (mean‐tokens) 을 사용했다는 의미입니다. 다시 말해서 NLI
데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.
SentenceTransformer 로 로드할 수 있는 다양한 모델에 대한 리스트는 아래의 링크에서 확인 가능합니
다.
해당 링크에 한국어 버전의 모델들 또한 공개되어 있으니 방문해보세요.
링크: https://huggingface.co/models?library=sentence‐transformers
데이터에서 모든 질문열. 즉, train_data[‘Q’] 에 대해서 문장 임베딩 값을 구한 후 embedding 이라는 새
로운 열에 저장합니다.

In [5]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis =1)

2개의 벡터로부터 코사인 유사도를 구하는 함수 cos_sim 를 정의합니다.

In [6]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

return_answer 함수는 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열.
즉, train_data[‘embedding’] 에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인
유사도 값이 가장 높은 질문 샘플을 찾아냅니다. 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴합
니다.

In [8]:
def return_answer(question):
  embedding = model.encode(question)
  train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'],
                                                         embedding), axis=1)
  return train_data.loc[train_data['score'].idxmax()]['A']

이제 챗봇을 테스트해봅시다.

In [9]:
return_answer('결혼 하고 싶어')

'좋은 사람이랑 결혼할 수 있을 거예요.'

In [10]:
return_answer('나랑 커피 먹을래?')

'카페인이 필요한 시간인가 봐요.'

In [11]:
return_answer('반가워')

'저도 반가워요.'

In [12]:
return_answer('너무 짜증나')

'짜증날 땐 짜장면'